# eVMI class dev
20/11/20
PH

Build on `tmoDataBase` class for VMI data:

- Constuct images
- BG & subtractions
- Masking
- Covariance

Previous work:

- [classDemo](https://pswww.slac.stanford.edu/jupyterhub/user/phockett/notebooks/dev/classDemo_191120.ipynb#)
- [Base class dev](https://pswww.slac.stanford.edu/jupyterhub/user/phockett/notebooks/dev/data_handling_dev_171120.ipynb)


## Setup

In [ ]:
import numpy as np

from h5py import File

from pathlib import Path

# HV imports
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

import xarray as xr


# Memory profiler - OPTIONAL for testing
# https://timothymonteath.com/articles/monitoring_memory_usage/
%load_ext memory_profiler
%memit


# Quick hack for slow internet connection!
%autosave 36000


# Import class - this requires pointing to the `tmoDataBase.py` file.
# See https://github.com/phockett/tmo-dev

# Import class from file
import sys
sys.path.append(r'/cds/home/p/phockett/dev/tmo-dev')
import tmoDataBase as tb

tb.setPlotDefaults()  # Set plot defaults (optional)

## Read data

In [ ]:
# Set main data path
baseDir = Path('/reg/data/ana15/tmo/tmolw0618/scratch/preproc/v2')

# Setup class object and which runs to read.
data = tb.tmoDataBase(fileBase = baseDir, runList = range(89,97+1), fileSchema='_preproc_elecv2')

# The class has a bunch of dictionaries holding info on the data
# data.runs['files']

# Read in the files with h5py
# There is very basic checking implemented currently, just to see if 'energies' is present.
data.readFiles()

## eVMI class dev

- v 23/11/20 Basically working now with multiplefilter case, but possibly some issues with image renorm, and generally very ugly!

In [ ]:
# Dev code for new class
# Inherit from base class, just add evmi functionality here
from scipy.ndimage import gaussian_filter

class VMI(tb.tmoDataBase):
    
    def __init__(self, **kwargs):
        # Run __init__ from base class
        super().__init__(**kwargs)

        # Filter update - add multilevel filtering here.
        # SHOULD propagate back to base class, but keep here for now as they're only applied to image processing.
        # Set some default filters. Use self.setFilter() to change.
        self.filters = {'signal':{'gas':True,
                                  'desc': 'Signal filter.'},
                        'bg':{'gas':False,
                              'desc': 'Background filter.'},
                        }

    def filterData(self, filterOptions = {}, keys = None, dim = 'energies'):
        """Wrapper for filterData when using nested filter (v2, 23/11/20)"""

        # Update filters if required
        if filterOptions:
            self.setFilter(filterOptions)

        # Default to all datasets
        if keys is None:
            keys = self.runs['proc']

        # Loop over filter sets and pass to base filterData() method.
        for key in self.filters.keys():
#             print(self.filters[key])
            super().filterData(filterOptions = self.filters[key], keys = keys, dim = dim)

            # Sort outputs to nested format
            # Note this leaves current settings in self.data[key]['mask']
            # These are STILL USED by histogram functions
            for runKey in keys:
                if key not in self.data[runKey].keys():
                    self.data[runKey][key] = {}  # Init
  

# REMOVED since it's confusing - will always leave last filter mask set!
#                 self.data[runKey][key]['mask'] = self.data[runKey]['mask'].copy()
#                 self.data[runKey][key]['filter'] = self.filters[key]
                
                
    # 1st go... running, but very slow.
    # Would probably be faster to write this all for np.arrays, rather than using existing image2d.
    def genVMI(self, bgSub=True, norm=True, keys=None, filterOptions={}, **kwargs):
        """Generate VMI images from event data, very basic hv.Image version."""
#         Quick test for run 89 - looks to be working, different images for each case.

#         Need to improve:

#         - Cmapping (maybe log10?)
#         - Image processing, use gaussian kernel?
#         - Move to Xarray dataset image stack for more advanced/careful processing...?

# %%timeit
# 24.3 s ± 370 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) LW06, Runs 89 - 97 (good only)
        
        # Default to all datasets
        if keys is None:
            keys = self.runs['proc']
        
        # Use existing image2d to generate images.
        # This is possibly a bit slow, may be faster to rewrite 2D hist code (see old CIS codes for fast methods)
        self.image2d(dim=['xc','yc'], filterOptions=filterOptions)  # keys=keys,
        
        # Restack
        self.eVMI = {'full':{}, 'fullNorm':{}, 'bg':{}, 'bgNorm':{}, 'bgSub':{}}  # Define output dicts
        for key in keys:
            self.eVMI['full'][key] = self.data[key]['img']
            
            # Norm by no. events with gas.
            # May want to norm by other factors too?
            self.eVMI['fullNorm'][key] = self.data[key]['img'].transform(z=hv.dim('z')/np.array(self.data[key]['raw']['gas']).sum())
            
        # Background images - assumed to be same filter(s) but gas off
        # NOTE: MAY NEED TO USE EXPLICT .copy() here? TBC
        if bgSub:
            filterOptions['gas'] = [False]  # Hard coded for now, should allow passing for more flexibility
            self.image2d(dim=['xc','yc'], filterOptions=filterOptions)  # keys=keys,
            
            for key in keys:
                self.eVMI['bg'][key] = self.data[key]['img']
                self.eVMI['bgNorm'][key] = self.data[key]['img'].transform(z=hv.dim('z')/(~np.array(vmi.data[key]['raw']['gas']).astype(bool)).sum())
        
                # Direct data manipulation OK, returns numpy.ndarray
                # Seems easiest way to go for now.
                # But might be better with hv.transform methods?
                # Didn't need dim in testing... but did here. Odd!
                self.eVMI['bgSub'][key] = hv.Image(self.eVMI['fullNorm'][key].data['z'] - self.eVMI['bgNorm'][key].data['z'])
                
    
    def genVMIXmulti(self, filterOptions={}, **kwargs):
        """Wrapper for genVMIX with multiple filter sets."""
        
        if filterOptions is not None:
            self.setFilter(filterOptions = filterOptions)
        
        # Run genVMIX for each filter set
        # Note bgSub = False to avoid recursive run in current form (v2), but should update this
        for item in self.filters.keys():
            if self.verbose['main']:
                print(f'Generating VMI images for filters: {item}')
            
            # Pass only single filter set here.
            # Should change to avoid repetition of filtering.
            self.genVMIX(bgSub=False, name=item, filterOptions = self.filters[item], **kwargs)
            
        
    
    # 2nd go, stack to Xarrays for processing        
    def genVMIX(self, bgSub=True, norm=True, keys=None, filterOptions={}, 
                bins = (np.arange(0, 1048.1, 1)-0.5,)*2, dim=['yc','xc'], name = 'imgStack', **kwargs):
        """Generate VMI images from event data, very basic Xarray version.
        
        v2: allow for multi-level filter via genVMIXmulti wrapper, changed to super() for filter.
            TODO: clean this up, currently using a nasty mix of new and old functionality.
                  Also issues with ordering of functions, and whether some dicts. are already set.
                  (Should filter all, then genVMIX.)
        
        v1: single filter set with hard-coded, recursive bg subtraction.
        """
 
        # %%timeit
        # 18.8 s ± 63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) LW06, Runs 89 - 97 (good only)

        # Default to all datasets
        if keys is None:
            keys = self.runs['proc']
            
        if filterOptions is not None:
#             print(filterOptions)
#             self.filterData(filterOptions = filterOptions)
            super().filterData(filterOptions = filterOptions)  # Use super() for case of single filter set.
        
        # Current method below (as per Elio's code). For LW06 run 89 tests (~70k shots, ~7M events)
        # Single shot: 1.88 s ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
        # All shots: 9.2 s ± 62.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
        # All shots, convert to int: 11.5 s ± 399 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

        # See also psana, opal.raw.image(evt)
        
        # Loop over all datasets
        imgArray = np.empty([bins[0].size-1, bins[1].size-1, len(keys)])  # Set empty array
        normVals = []
        metrics = {'filterOptions':filterOptions.copy()}  # Log stuff to Xarray attrs
        
        for n, key in enumerate(keys):
            # Initially assume mask can be used directly, but set to all True if not passed
            # Will likely want more flexibility here later
#             if mask is None:
#             mask = np.ones_like(self.data[key]['raw'][dim[0]]).astype(bool)

            # Check mask exists, set if not
            if 'mask' not in self.data[key].keys():
#                 self.filterData(keys=[key])
                super().filterData(keys=[key])  # Use super() for case of single filter set.

            # Note flatten or np.concatenate here to set to 1D, not sure if function matters as long as ordering consistent?
            # Also, 1D mask selection will automatically flatten? This might be an issue for keeping track of channels?
            # Should use numpy masked array...? Slower in testing.
            d0 = np.array(self.data[key]['raw'][dim[0]])[self.data[key]['mask']].flatten()
            d1 = np.array(self.data[key]['raw'][dim[1]])[self.data[key]['mask']].flatten()

            # Stack to np array
            imgArray[:,:,n] = np.histogram2d(d0,d1, bins = bins)[0]
            
            metrics[key] = {'shots':self.data[key]['raw'][dim[0]].shape,
                            'selected':self.data[key]['mask'].sum(),
                            'gas':np.array(self.data[key]['raw']['gas']).sum(),
                            'events':d0.size,
                            'norm':self.data[key]['mask'].size}
            
            normVals.append(self.data[key]['mask'].size) # shots selected - only for norm to no gas?
            
            if name not in self.data[key].keys():
                self.data[key][name] = {}
                
            self.data[key][name]['metrics'] =  metrics[key].copy() # For mult filter case, push metrics to filter dict.
            self.data[key][name]['mask'] = self.data[key]['mask'].copy()
            
#         return imgArray
        # Convert to Xarray
#         imgStack = xr.DataArray(imgArray, dims=[dim[0],dim[1],'run'], 
#                                 coords={dim[0]:bins[0][0:-1], dim[1]:bins[1][0:-1], 'run':keys},
#                                 name = 'imgStack')
        # 2nd attempt, swap dim labels & reverse y-dir. This maintains orientation for image plots.
        imgStack = xr.DataArray(imgArray, dims=[dim[0],dim[1],'run'], 
                                coords={dim[0]:bins[0][:-1], dim[1]:bins[1][-2::-1], 'run':keys},
                                name = name)

        imgStack['norm'] = ('run', normVals)  # Store normalisation values
        
        if norm:
            imgStack = imgStack/imgStack['norm']
            imgStack.name = name  # Propagate name! Division kills it
        
        imgStack.attrs['metrics'] = metrics
        
        # Recursive call for bg calculation, but set bgSub=False
        # CURRENTLY NOT WORKING - always get idential (BG only) results for both cases???
        # As constructed ALWAYS assumes 1st call will have bgSub = True
#         if bgSub:
#             self.imgStack = imgStack.copy()  # May need .copy() here?
#             filterOptions['gas'] = [False]  # Set bg as gas off, all other filter options identical
#             self.genVMIX(bgSub=False, norm=norm, keys=keys, filterOptions=filterOptions, bins = bins, **kwargs)
#         else:
#             self.imgStackBG = imgStack.copy().rename('BG')

        # Try keeping multiple results sets in stack instead.
        # This is a little ugly, but working.
        if not hasattr(self,'imgStack'):
            # self.imgStack = []  # May need .copy() here?  # v1, set as list
            self.imgStack = xr.Dataset()  # v2, set as xr.Dataset and append Xarrays to this
        
#         self.imgStack.append(imgStack.copy())  # May need .copy() here?  # v1
        self.imgStack[name] = imgStack.copy()  # v2 using xr.Dataset - NOTE THIS KILLS METRICS!
                                        # TODO: push to main dict, or coord?
        
        
        if bgSub:
            filterOptions['gas'] = [False]  # Set bg as gas off, all other filter options identical
            self.genVMIX(bgSub=False, norm=norm, name=name+'BG', keys=keys, filterOptions=filterOptions, bins = bins, **kwargs)
#             self.imgStack.append((self.imgStack[-2] - self.imgStack[-1]).rename(name + 'BGsub'))  # Use last 2 img sets for subtraction
            self.imgStack[name + 'BGsub'] = ((self.imgStack[name] - self.imgStack[name+'BG']).rename(name + 'BGsub'))
        
        # Restack final output to NxNxm Xarray for easy manipulation/plotting.
#         self.imgStack = self.imgStack.to_array(dim = 'type').rename('stacked')
    
#     def imgStacksub(self)
    
    # TODO: want to chain this for image plotting, but first set which array to use!
    # TODO: options for which dataset to use, just hacked in duplicate code for now.
    def restackVMIdataset(self):
        # Restack image dataset to NxNxm Xarray for easy manipulation/plotting.
        # Return rather than set internally...?
  
        if hasattr(self, 'imgReduce'):
        # Restack, note transpose to force new dim ('type') to end.
            # This currently matters for smoothing function with scipy gaussian_filter.
            imgReduce = self.imgReduce.to_array(dim = 'type').rename('stacked')
            #.transpose('yc','xc','run','type')

            # Send new dim to end
            dimStack = imgReduce.dims
        #         self.imgStack = self.imgStack.transpose(*dimStack[1:],dimStack[0])
            return imgReduce.transpose(*dimStack[1:],dimStack[0])
        
        else:
            # Restack, note transpose to force new dim ('type') to end.
            # This currently matters for smoothing function with scipy gaussian_filter.
            imgStack = self.imgStack.to_array(dim = 'type').rename('stacked')
            #.transpose('yc','xc','run','type')

            # Send new dim to end
            dimStack = imgStack.dims
        #         self.imgStack = self.imgStack.transpose(*dimStack[1:],dimStack[0])
            return imgStack.transpose(*dimStack[1:],dimStack[0])

        
    def downsample(self, step = [2,2], dims = ['xc','yc']):
        """Wrapper for xr.coarsen to downsample images by step.
        
        Set to trim boundaries, and sum over points. Coord system will be maintained.
        
        This will work for Dataset or Dataarray forms.
        Currently set to use smoothed dataset if available, or imgStack if not.
        
        TODO: add some options here.
        """
        
        # v1 with list
#         self.imgReduce = []
        
#         for n, item in enumerate(self.imgStack):
# #             print(n)
#             self.imgReduce.append(item.coarsen({dim[0]:step[0], dim[1]:step[1]}, boundary="trim").sum())
# #             self.imgReduce[n] = item.coarsen({dim[0]:step[0], dim[1]:step[1]}, boundary="trim", keep_attrs=True).sum()
    
        # v2 with DataArray
#         self.imgReduce = self.imgStack.coarsen({dims[0]:step[0], dims[1]:step[1]}, boundary="trim").sum()
#         for item in ['imgStack', 'imgSmoothed']:
        if hasattr(self, 'imgSmoothed'):
            self.imgReduce = self.imgSmoothed.coarsen({d:s for (d,s) in zip(dims,step)}, boundary="trim").sum()
        else:
            self.imgReduce = self.imgStack.coarsen({d:s for (d,s) in zip(dims,step)}, boundary="trim").sum()
    
    
    def smooth(self, sigma = [1,1]):
    # Try using scipy.ndimage for smoothing...
    # Can use xr.apply_ufunc for this.
    # NOTE: this applies to ALL DIMS, so set 0 for additional stacking dims!
    # NOTE: this currently assumes dim ordering (not checked by name)
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html
    
    # smoothed = xr.apply_ufunc(gaussian_filter, imgReduce, 1)

    # v1 for lists
#         self.imgSmoothed = []

#         # TODO: add options for which stack to smooth
#         for item in enumerate(self.imgStack):
#             self.imgSmoothed.append(xr.apply_ufunc(gaussian_filter, item, sigma))  # Final value is sigma [dim0,dim1...]
    
        # v2 with DataArray
        # Set any additional dims to zero
        if len(sigma) != self.imgStack.ndims:
            sigma = np.pad(sigma, [0, self.imgStack.ndims - len(sigma)])
            
        self.imgSmoothed = (xr.apply_ufunc(gaussian_filter, self.imgStack, sigma))  # Final value is sigma [dim0,dim1...]
    
#     def fastHist(self, dims = ['xc','yc'], bins = [1048,1048], nullEvent = -9999):
#         """Generate 2D histogram via indexing. (Fast for small batches, but needs work for scale-up.)
        
#         NOTE: currently set for electron images, 1048x1048 bins.
#         To explore: see notes below!
#         """
        
#         hist2D = np.zeros(bins+2)  # Sized for 1-indexed bins, plus final cell for invalid hits.

#         # Index method... WAY FASTER single shot (orders of magnitude on np.histogram2d), moderately faster (~30%) all shots (but should be able to improve with Numba?)
#         # Lots of options here, for now use .flatten() to allow for np.delete below. Masking may be faster?
#         d0 = np.array(self.data[key]['raw'][dims[0]]).astype(int).flatten() 
#         d1 = np.array(self.data[key]['raw'][dims[1]]).astype(int).flatten()
        
#         #  easy way to drop -9999 "no hits"?
#         d0 = np.delete(xhits, np.where(xhits == -9999))  # This only works for 1D case! May want to set NaNs instead?
#         d1 = np.delete(yhits, np.where(yhits == -9999))

In [ ]:
vmi = VMI(fileBase = baseDir, runList = range(89,97+1), fileSchema='_preproc_elecv2')

In [ ]:
vmi.readFiles()

In [ ]:
# %%timeit
# 24.3 s ± 370 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) LW06, Runs 89 - 97 (good only)
# vmi.genVMI()

## Xarray img stack tests

In [ ]:
# %%timeit
# 18.8 s ± 63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) LW06, Runs 89 - 97 (good only)
# 19.8 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) LW06, Runs 89 - 97 (good only), 23/11/20 version
# vmi.genVMIX(filterOptions = {'gas':[True]})
dims = ['yc','xc']
vmi.genVMIXmulti(dims=dims)

In [ ]:
len(vmi.imgStack)  # == vmi.imgStackBG

In [ ]:
vmi.imgStack

In [ ]:
# Subtraction...
vmi.imgStack['sub'] = vmi.imgStack['signal'] - vmi.imgStack['bg']
vmi.imgStack

In [ ]:
vmi.imgStack['signal'].max()

In [ ]:
vmi.imgStack['signal'].norm

In [ ]:
vmi.imgStack['bg'].norm

In [ ]:
vmi.imgStack['bg'].max()

In [ ]:
vmi.imgStack['signal'].size

In [ ]:
# Currently setting duplicate array... not sure why? Missing .copy() somewhere... but was previously working (single filter case)
(vmi.imgStack['bg'] == vmi.imgStack['signal']).sum()

In [ ]:
vmi.data[89]['signal']

In [ ]:
vmi.data[89]['bg']

In [ ]:
# Bug with metrics somewhere...?
vmi.data[89]['signal']['metrics'] == vmi.data[89]['bg']['metrics']

In [ ]:
vmi.data[89]['signal']['mask'].sum()

In [ ]:
# vmi.restackVMIdataset()

In [ ]:
vmi.imgStack

In [ ]:
# TO FIX - something still not right here... metrics shouldn't be the same, although 'events' seems OK.
# Inadvertently setting pointer somewhere...
vmi.imgStack[0].metrics

In [ ]:
vmi.imgStack[1].metrics

In [ ]:
vmi.imgStack[1].norm

In [ ]:
# (vmi.imgStackBG*vmi.imgStackBG.norm).sel(run=89).sum()

In [ ]:
# (vmi.imgStack*vmi.imgStack.norm).sel(run=89).sum()

In [ ]:
step = [2,2]
dims = ['xc','yc']

# list(zip(dims,step))
{d:s for (d,s) in zip(dims,step)}

In [ ]:
# Try downsample & plot
vmi.downsample(step=[5,5])

In [ ]:
vmi.imgReduce

In [ ]:
# imgReduce = []
# step=[2,2]

# for n, item in enumerate(vmi.imgStack):
#     print(n)
# #     imgReduce[n] = 
#     test = item.coarsen({dim[0]:step[0], dim[1]:step[1]}, boundary="trim").sum()

In [ ]:
vmi.restackVMIdataset()

In [ ]:
vmi.restackVMIdataset().dims

In [ ]:
dims

In [ ]:
# TODO: change to dict for easy pass & layout!
fSize=[500,500]
# dims = ['yc','xc']
# hv.Image(vmi.imgReduce.sel(run=89)).opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.15))
hv.Image(vmi.restackVMIdataset().sel(run=89, type='signal'), kdims = dims).opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.15))

In [ ]:
imgPlot = hv.Image(vmi.restackVMIdataset().sel(run=89, type='sub'), kdims = dims).opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.15))
(imgPlot.hist() + imgPlot.sample(xc=500) + imgPlot.sample(yc=552)).cols(1)
# (imgPlot.hist() * imgPlot.sample(xc=512))

In [ ]:
vmi.restackVMIdataset().sel(run=89, type='sub')

In [ ]:
# imgPlot[450:650:5,400:600:5,...].to.curve().overlay().opts(tools=['hover'])  # Stack curves - hover not working?
# imgPlot[450:650:5,400:600:5,...].to.curve()  # xc slices
# imgPlot[450:650:5,400:600:5,...].to.curve(kdims='xc')  # yc slices
# imgPlot[450:650:5,400:600:5,...].to.curve() + imgPlot[450:650:5,400:600:5,...].to.curve(kdims='xc')  # Nope
# hv.Curve(imgPlot[450:650:5,400:600:5,...], kdims='xc', vdims='stacked')  # Nope
# hv.Curve(vmi.restackVMIdataset().sel(run=89, type='sub')[450:650:5,400:600:5]) # Nope

In [ ]:
imgPlot.sample(xc=503) * imgPlot.sample(yc=553)

In [ ]:
hv.Curve(imgPlot.sample(xc=np.arange(500,503)), kdims=['yc','stacked'], vdims='xc') # * imgPlot.sample(yc=553)

In [ ]:
sampled = imgPlot.apply.sample(xc=np.arange(500,503)) #.collapse()
hv.Curve(sampled)

In [ ]:
imgPlot.sample(xc=np.arange(500,503)).layout()

In [ ]:
testRestackHV = hv.Dataset(vmi.restackVMIdataset())
testRestackHV.to(hv.Image, kdims=['xc','yc']).opts(colorbar=True)

In [ ]:
fSize=[500,500]
hv.Image(vmi.imgReduce[1].sel(run=89)).opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.15))

In [ ]:
hv.Image(vmi.imgReduce[2].sel(run=89)).opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.15))

In [ ]:
vmi.imgStack[0].sum()

In [ ]:
# This is nice with slider, but range setting not working here?
hv_ds = hv.Dataset(vmi.imgReduce[2].rename('downsampled'))
hvImg = hv_ds.to(hv.Image, kdims=["yc", "xc"]).redim.range(z=(0, 0.15)) #, dynamic=False)

In [ ]:
hvImg #.redim.range(z=(0, 0.15))

In [ ]:
import xarray as xr

bins = (np.arange(0, 1048.1, 1)-0.5,)*2
imgStack = xr.DataArray(testX, dims=['x','y','run'], coords={'x':bins[0][0:-1], 'y':bins[1][0:-1], 'run':vmi.runs['proc']})

In [ ]:
norm = [1,2,3,4]
imgStack['norm'] = ('run', norm)

## Image gen method tests

In [ ]:
# Quick single-shot image test...
key = 89
shot = 0
# display(hv.Curve(vmi.data[key]['raw']['xc'][shot,:]))
# display(hv.Curve(vmi.data[key]['raw']['yc'][shot,:]))

In [ ]:
vmi.data[key]['raw']['xc'][shot,:]

In [ ]:
vmi.data[key]['raw']['yc'][shot,:]

In [ ]:
vmi.data[key]['raw']['yc'][shot,:].max()

In [ ]:
# %%timeit
# 1.88 s ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# Current method (Elio's code)
# See also psana, opal.raw.image(evt)
dim = ['xc','yc']
bins = (np.arange(0, 1048.1, 1)-0.5,)*2
    
d0 = np.array(vmi.data[key]['raw'][dim[0]])[shot,:] #[self.data[key]['mask']].flatten()
d1 = np.array(vmi.data[key]['raw'][dim[1]])[shot,:] #[self.data[key]['mask']].flatten()

img2d = np.histogram2d(d0,d1, bins = bins)[0]
# hv.Image((np.histogram2d(d0,d1, bins = bins)[0]), dim).redim

In [ ]:
bins = (np.arange(0, 1048.1, 1)-0.5,)*2
# bins[0].size
keys = [89, 91, 93, 97]
imgArray = np.empty([bins[0].size, bins[1].size, len(keys)])

In [ ]:
imgArray.shape

In [ ]:
%%timeit
# Try with ints - should be faster...? ACTUALY SEEMS to be slower by several s, in this form at least.
# https://numpy.org/devdocs/reference/generated/numpy.histogram2d.html#numpy.histogram2d
# Current method (Elio's code)
# See also psana, opal.raw.image(evt)
dim = ['xc','yc']
# bins = (np.arange(0, 1048.1, 1)-0.5,)*2
# bins = (np.arange(0, 1048))*2
bins = [1048, 1048]
binRange = [[0,1048],[0,1048]]
    
d0 = np.array(vmi.data[key]['raw'][dim[0]]).astype('int').flatten() #[self.data[key]['mask']].flatten()
d1 = np.array(vmi.data[key]['raw'][dim[1]]).astype('int').flatten() #[self.data[key]['mask']].flatten()

img2d, xe, ye = np.histogram2d(d0,d1, bins = bins, range = binRange)

In [ ]:
[img2d.sum(), img2d.max()]

In [ ]:
xe

In [ ]:
bins[0].size

In [ ]:
vmi.data[key]['raw']['xc'][shot,:].ndim

In [ ]:
# %%timeit
# 8.81 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

# Try index method... WAY FASTER
test2d = np.zeros([1048,1048])

# Try indexing method - easy way to drop -9999 "no hits"?
xhits = np.array(vmi.data[key]['raw']['xc'][shot,:]).astype(int)
yhits = np.array(vmi.data[key]['raw']['yc'][shot,:]).astype(int)

xhits = np.delete(xhits, np.where(xhits == -9999))
yhits = np.delete(yhits, np.where(yhits == -9999))

test2d[xhits, yhits] += 1

In [ ]:
img2d.shape

In [ ]:
test2d.shape

In [ ]:
(img2d-test2d).sum()

In [ ]:
# %%timeit
# Single shot: 1.88 s ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots: 9.2 s ± 62.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots, convert to int: 11.5 s ± 399 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# Current method (Elio's code)
# See also psana, opal.raw.image(evt)
dim = ['xc','yc']
bins = (np.arange(0, 1048.1, 1)-0.5,)*2
    
# d0 = np.array(vmi.data[key]['raw'][dim[0]])[shot,:] #[self.data[key]['mask']].flatten()
# d1 = np.array(vmi.data[key]['raw'][dim[1]])[shot,:] #[self.data[key]['mask']].flatten()
d0 = np.array(vmi.data[key]['raw'][dim[0]]).flatten()
d1 = np.array(vmi.data[key]['raw'][dim[1]]).flatten()


img2d = np.histogram2d(d0,d1, bins = bins)[0]
# hv.Image((np.histogram2d(d0,d1, bins = bins)[0]), dim).redim

In [ ]:
# %%timeit
# Single shot: 8.81 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# All shots 2D: 6.64 s ± 34.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots with flatten() when indexing: 7.46 s ± 68.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots with flatten() plus delete: 6.91 s ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots 2D with masking: 7.82 s ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots 2D with masking + compressed() index: 7.44 s ± 62.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots 2D with boolean mult index: 5.56 s ± 103 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Try index method... WAY FASTER single shot, moderately faster all shots (but should be able to improve with Numba?)
test2d = np.zeros([1050,1050])

# Try indexing method - easy way to drop -9999 "no hits"?
# xhits = np.array(vmi.data[key]['raw']['xc'][shot,:]).astype(int)
# yhits = np.array(vmi.data[key]['raw']['yc'][shot,:]).astype(int)
xhits = np.array(vmi.data[key]['raw']['xc']).astype(int)  #.flatten()
yhits = np.array(vmi.data[key]['raw']['yc']).astype(int)  #.flatten()

# xhits = np.delete(xhits, np.where(xhits == -9999))  # This only works for 1D case! May want to set NaNs instead?
# yhits = np.delete(yhits, np.where(yhits == -9999))
# xhits[np.where(xhits == -9999)] = np.nan  # nans also throw an error
# yhits[np.where(yhits == -9999)] = np.nan
# xhits[np.where(xhits == -9999)] = 1049  # Reindex to temp bin, then remove later
# yhits[np.where(yhits == -9999)] = 1049

# Try masked array
# xhits = np.ma.masked_equal(xhits, -9999)  # Creates mask, but doesn't work directly as index - still need to flatten?
# yhits = np.ma.masked_equal(yhits, -9999)


# test2d[xhits, yhits] += 1
# test2d[np.ma.filled(xhits,1049), np.ma.filled(yhits,1049)] += 1
# test2d[xhits.compressed(), yhits.compressed()] += 1
# test2d[xhits.flatten(), yhits.flatten()] += 1
# test2d[~(xhits == -9999) * xhits, ~(yhits == -9999) * yhits] += 1   # This seems to work, but only uses 1st col to index? Miss A LOT of events.
test2d[~(xhits == -9999) * xhits, ~(yhits == -9999) * yhits] = test2d[~(xhits == -9999) * xhits, ~(yhits == -9999) * yhits] + 1  # SAME RESULT
# AH - probably due to inplace editing with repeated indicies. May need to accumulate or loop in 1 dim? Ugh.
# MIGHT return to np.histogram2d for now!
# test2d[(~(xhits == -9999) * xhits).flatten(), (~(yhits == -9999) * yhits).flatten()] += 1  

In [ ]:
[(~(xhits == -9999)).sum(), (~(yhits == -9999)).sum()]  # * yhits

In [ ]:
# Note a lot of missing elements here!
test2d.sum()

In [ ]:
img2d.sum()

In [ ]:
xhits = np.array(vmi.data[key]['raw']['xc']).astype(int) 
# xhits = np.ma.masked_equal(xhits, -9999)
# xhits.min()
~(xhits == -9999) * xhits

In [ ]:
# xhits = np.array(vmi.data[key]['raw']['xc'])
# hv.Curve((xhits > 0).sum(axis = 1)[0:1000])  # Events per shot
# np.sum(np.where(xhits > 0),0)

Note many shots top out (detector saturated). May want to use this as a filter parameter?

## 2D hist Numba tests - needs work!

In [ ]:
# TRY AS NUMBA FUNCTION...
# Single shot: 8.81 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# All shots 2D: 6.64 s ± 34.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots with flatten() when indexing: 7.46 s ± 68.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots with flatten() plus delete: 6.91 s ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots 2D with masking: 7.82 s ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# All shots 2D with masking + compressed() index: 7.44 s ± 62.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

from numba import jit, njit, guvectorize

# This is poor, since it just uses object mode
# timeit same as raw function: 6.77 s ± 52.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# @jit
# def numba2dhist(xhits, yhits):
#     # Try index method... WAY FASTER single shot, moderately faster all shots (but should be able to improve with Numba?)
#     test2d = np.zeros([1050,1050])
#  REST AS PREVIOUSLY

@jit(parallel = True)
def numba2dhist(xhits, yhits, test2d):
    # Try index method... WAY FASTER single shot, moderately faster all shots (but should be able to improve with Numba?)
#     test2d = np.zeros([1050,1050])

    # Try indexing method - easy way to drop -9999 "no hits"?
    # xhits = np.array(vmi.data[key]['raw']['xc'][shot,:]).astype(int)
    # yhits = np.array(vmi.data[key]['raw']['yc'][shot,:]).astype(int)
#     xhits = np.array(vmi.data[key]['raw']['xc']).astype(int)  #.flatten()
#     yhits = np.array(vmi.data[key]['raw']['yc']).astype(int)  #.flatten()

    # xhits = np.delete(xhits, np.where(xhits == -9999))  # This only works for 1D case! May want to set NaNs instead?
    # yhits = np.delete(yhits, np.where(yhits == -9999))
    # xhits[np.where(xhits == -9999)] = np.nan  # nans also throw an error
    # yhits[np.where(yhits == -9999)] = np.nan
    # xhits[np.where(xhits == -9999)] = 1049  # Reindex to temp bin, then remove later
    # yhits[np.where(yhits == -9999)] = 1049

    # Try masked array
    xhits = np.ma.masked_equal(xhits, -9999)  # Creates mask, but doesn't work directly as index - still need to flatten?
    yhits = np.ma.masked_equal(yhits, -9999)


    # test2d[xhits, yhits] += 1
    # test2d[np.ma.filled(xhits,1049), np.ma.filled(yhits,1049)] += 1
    test2d[xhits.compressed(), yhits.compressed()] += 1
    # test2d[xhits.flatten(), yhits.flatten()] += 1
    
# As vectorized kernel... needs work!
# @guvectorize([(int[:,:], int[:,:], int[:,:])], '(n,n),(n,n)->(m,m)')
# def numba2dhist(xhits, yhits, output):
#     xhits[np.where(xhits == -9999)] = 1049  # Reindex to temp bin, then remove later
#     yhits[np.where(yhits == -9999)] = 1049
    
#     test2d[xhits, yhits] += 1

In [ ]:
# %%timeit

# xhits = np.array(vmi.data[key]['raw']['xc']).astype(int)  #.flatten()
# yhits = np.array(vmi.data[key]['raw']['yc']).astype(int)  #.flatten()

# test2d = np.zeros([1050,1050])

# # same as raw function: 6.77 s ± 52.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# numba2dhist(xhits, yhits, test2d)

# # TODO: try alternative codes, also guvectorise etc.

In [ ]:
# If using masking, this will show hits per shot
np.ma.clump_unmasked(xhits) #.compressed()

## Image gen working (v1 with hv.Image)

In [ ]:
fSize = [600,600]
# vmi.eVMI['bgSub'][89].opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.005))

In [ ]:
fSize = [600,600]

# This times out...?
for key in vmi.eVMI.keys():
    print(key)
    
    if key.endswith('Norm'):
        display(vmi.eVMI[key][89].opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.005)))

In [ ]:
vmi.eVMI['bgSub'][89].opts(height=fSize[0], aspect='square').redim.range(z=(0, 0.005))

Quick test for run 89 - looks to be working, different images for each case.

Need to improve:

- Cmapping (maybe log10?)
- Image processing, use gaussian kernel?
- Move to Xarray dataset image stack for more advanced/careful processing...?

In [ ]:
vmi.filterOptions

In [ ]:
key=89
vmi.eVMI['fullNorm'][key]

In [ ]:
vmi.eVMI['bgNorm'][key]

In [ ]:
hv.Image(vmi.eVMI['fullNorm'][key].data['z'] - vmi.eVMI['bgNorm'][key].data['z'])

In [ ]:
vmi.data[89]['img']

In [ ]:
dir(vmi.data[89]['img'])

In [ ]:
# Direct data manipulation OK, returns numpy.ndarray
# Seems easiest way to go for now.
# May need dim too?
testSub = vmi.data[89]['img'].data - vmi.data[89]['img'].data

# Doesn't work with dataset
# testSub = vmi.data[89]['img'].dataset - vmi.data[89]['img'].dataset  # FAILS

In [ ]:
type(testSub)

In [ ]:
hv.Image(testSub)

In [ ]:
# Check dataset org
vmi.data[89]['img'].dataset

In [ ]:
# Test self subtraction - OK
# testSub = vmi.data[89]['img'].transform(z=hv.dim('z')-hv.dim('z'))

# Test subtraction of another hv dataset - NOT OK... returns something, but not correct
# testSub = vmi.data[89]['img'].transform(z=hv.dim('z')-vmi.data[89]['img'].data) #['z'])

# With dim - NOT OK, throws errors
# testSub = vmi.data[89]['img'].transform(z=hv.dim('z')-vmi.data[89]['img'].data['z'])

# With dim - NOT OK, throws errors
# testSub = vmi.data[89]['img'].transform(z=hv.dim('z')-vmi.data[89]['img'].dim('z'))
# testSub = vmi.data[89]['img'].transform(z=hv.dim('z')-vmi.data[89]['img'].dataset['z']) # Returns 1D array

In [ ]:
testSub.data['z'].max()

In [ ]:
testNorm = vmi.data[89]['img'].data

In [ ]:
vmi.data[89]['img'].dataset['z'].size

## Tests with HV image transforms
May use these, or just address data directly...

See http://holoviews.org/user_guide/Transforming_Elements.html

In [ ]:
hv.output(backend='matplotlib', size=200)

from scipy.misc import ascent

stairs_image = hv.Image(ascent()[200:500, :], bounds=[0, 0, ascent().shape[1], 300], label="stairs")
stairs_image

In [ ]:
stairs_image.transform(z=hv.dim('z')-(hv.dim('z')*10))

In [ ]:
# stairs_image.reduce(x=np.mean)
stairs_image.reduce(x=np.subtract)

In [ ]:
from scipy import ndimage

class image_filter(hv.Operation):
    
    sigma = param.Number(default=5)
    
    type_ = param.String(default="low-pass")

    def _process(self, element, key=None):
        xs = element.dimension_values(0, expanded=False)
        ys = element.dimension_values(1, expanded=False)
        
        # setting flat=False will preserve the matrix shape
        data = element.dimension_values(2, flat=False)
        
        if self.p.type_ == "high-pass":
            new_data = data - ndimage.gaussian_filter(data, self.p.sigma)
        else:
            new_data = ndimage.gaussian_filter(data, self.p.sigma)
        
        label = element.label + " ({} filtered)".format(self.p.type_)
        # make an exact copy of the element with all settings, just with different data and label:
        element = element.clone((xs, ys, new_data), label=label)
        return element

stairs_map = hv.HoloMap({sigma: image_filter(stairs_image, sigma=sigma)
                         for sigma in range(0, 12, 1)}, kdims="sigma")

stairs_map.opts(framewise=True)